In [1]:
import pandas as pd
import torch

In [2]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df

,POS_0_9_CP,POS_10_19_CP,POS_20_29_CP,POS_30_39_CP,POS_40_49_CP,POS_50_59_CP,POS_60_69_CP,POS_70_79_CP,POS_80_89_CP,POS_90_CP,DTH_CUM_CP
0,24,51,44,51,60,75,72,61,27,0,3
1,22,31,214,177,62,46,24,9,0,0,2
2,84,126,239,194,159,171,125,74,24,8,9
3,65,76,80,110,82,81,55,49,56,34,7
4,99,107,339,287,139,147,110,71,37,0,12
...,...,...,...,...,...,...,...,...,...,...,...
1039,86,201,206,202,240,317,225,138,48,0,7
1040,80,102,125,124,104,71,70,36,6,0,6
1041,268,358,277,415,355,332,215,107,47,7,5
1042,81,116,90,145,118,113,70,36,9,0,2


In [3]:
df.values

array([[ 24,  51,  44, ...,  27,   0,   3],
       [ 22,  31, 214, ...,   0,   0,   2],
       [ 84, 126, 239, ...,  24,   8,   9],
       ...,
       [268, 358, 277, ...,  47,   7,   5],
       [ 81, 116,  90, ...,   9,   0,   2],
       [118, 156, 197, ...,   0,   0,   5]])

In [4]:
trainX = torch.tensor(df.loc[:,'POS_0_9_CP':'POS_90_CP'].values,dtype = torch.float64)
trainY = torch.tensor(df.loc[:,'DTH_CUM_CP'].values,dtype = torch.float64).reshape(-1,1)
testX = torch.tensor(test.loc[:,'POS_0_9_CP':'POS_90_CP'].values,dtype = torch.float64)
testY = torch.tensor(test.loc[:,'DTH_CUM_CP'].values,dtype = torch.float64).reshape(-1,1)
trainY

tensor([[3.],
        [2.],
        [9.],
        ...,
        [5.],
        [2.],
        [5.]], dtype=torch.float64)

In [5]:
trainY

tensor([[3.],
        [2.],
        [9.],
        ...,
        [5.],
        [2.],
        [5.]], dtype=torch.float64)

## Q1: about how many bytes does trainX consume?
### Ans: (64/8)bytes * (1043-0+1)rows * (10)columns = 8 * 1044 * 10 = 83520

In [6]:
#q1
trainX.element_size() * trainX.shape[0] * trainX.shape[1]

83520

## Q2: what is the biggest difference we would have any one cell if we used float16 instead of float64? (do not understand)

In [7]:
#q2
tmp = trainX.to(torch.float16)
tmp = tmp.to(torch.float64)
float(torch.max(trainX - tmp))

0.0

## Q3: is a CUDA GPU available on your VM?

In [8]:
#q3
ans = False
if torch.cuda.is_available():
    ans = True
ans

False

In [9]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

### Q4: what is the predicted number of deaths for the first census tract?

In [10]:
#q4
result = testX[0] @ coef
result.item()

9.844

### Q5: what is the average number of predicted deaths, over the whole testX dataset?

In [11]:
#q5
torch.mean(testX @ coef).item()

12.073632183908048

### Q6: first, what is y when x is a tensor containing 0.0?

In [12]:
#q6
def f(x):
    return x ** 2 - 8 * x + 19
x = torch.tensor(0.0,requires_grad = True)
y = f(x)
float(y)

19.0

### Q7: what x value minimizes y?

In [13]:
#q7
optimizer = torch.optim.SGD([x],lr = 0.1)
for epoch in range(100):
    y = f(x)
    y.backward()
    optimizer.step()
    optimizer.zero_grad()
float(x)

3.999999523162842

### Q8: what is the MSE (mean-square error) when we make predictions using this vector of zero coefficients?

In [14]:
#q8
coef = torch.zeros(trainX.shape[1],1,dtype = torch.float64)
ans = trainX @ coef
loss_fn = torch.nn.MSELoss()
float(loss_fn(ans,trainY))

197.8007662835249

### Q9: what is the MSE over the training data, using the coefficients resulting from the above training?

In [15]:
torch.manual_seed(544)
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size = 50, shuffle = True)

In [16]:
#q9
coef = torch.zeros((trainX.shape[1],1),requires_grad = True,dtype = torch.float64)
optimizer = torch.optim.SGD([coef],lr = 0.000002)
ans = 0
for epoch in range(500):
    for trainx, trainy in dl:
        predict = trainx @ coef
        ans = loss_fn(predict,trainy)
        ans.backward()
        optimizer.step()
        optimizer.zero_grad()
loss_fn(trainX @ coef, trainY).item()

26.8113940147193

### Q10: what is the MSE over the test data?

In [17]:
#q10
loss_fn(testX @ coef, testY).item()

29.05854692548551